In [1]:
import os
import time
import datetime
import pandas as pd

import numpy as np
import random
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler
from sklearn.model_selection import train_test_split

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel, TrainingArguments, Trainer, DataCollatorForLanguageModeling
import warnings
warnings.filterwarnings("ignore")

In [3]:

class TextDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data['Articulos'][idx]
        summary = self.data['Resumen'][idx]

        inputs = self.tokenizer.encode_plus(
            summary,
            text,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': inputs['input_ids'].squeeze()
        }

In [4]:

def load_summary_dataset(tokenizer):
    file_path = "bbc_news_es.csv"
    df = pd.read_csv(file_path)[['Resumen', 'Articulos']]
    df = df[list(-(df["Resumen"].duplicated()))]
    # df = df[[0, 5]]
    # df.columns = ['Resumen', 'Articulos']
    # df = df.sample(300, random_state=1)
    # df = df.reset_index().drop(["index"], axis=1)

   # Dividir en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(
        df['Articulos'].tolist(), df['Resumen'].tolist(), test_size=0.1, random_state=1)
    X_train, X_valid, y_train, y_valid = train_test_split(
        X_train, y_train, test_size=0.11, random_state=1)
    train_data = pd.DataFrame({'Articulos':list(X_train), 'Resumen':list(y_train)})
    test_data = pd.DataFrame({'Articulos':list(X_test), 'Resumen':list(y_test)})
    valid_data = pd.DataFrame({'Articulos':list(X_valid), 'Resumen':list(y_valid)})

    train_dataset = TextDataset(train_data, tokenizer, max_length=1024)
    valid_dataset = TextDataset(valid_data, tokenizer, max_length=1024)
    test_dataset = TextDataset(test_data, tokenizer, max_length=1024)

    return train_dataset, valid_dataset, test_dataset

In [4]:
## Load model and data
#--------------

# set model name
model_name = "gpt2-xl"
#seed
torch.manual_seed(42)

# load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained(model_name, bos_token='<startoftext>', 
            eos_token='<endoftext>', pad_token='<pad>')

model = GPT2LMHeadModel.from_pretrained('gpt2').cuda()
model.resize_token_embeddings(len(tokenizer))

# Cargar y dividir el dataset
train_dataset, valid_dataset, test_dataset = load_summary_dataset(tokenizer)

# Argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='results',
    num_train_epochs=10,
    logging_steps=10,
    save_strategy='epoch',
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='logs'
)

# Collator de datos para el modelado de lenguaje
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Entrenamiento del modelo
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=data_collator
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


KeyboardInterrupt: 

In [5]:

# Cargar el modelo preentrenado y el tokenizer
model_name = 'DeepESP/gpt2-spanish'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Cargar y dividir el dataset en conjuntos de entrenamiento y prueba
train_dataset, valid_dataset, test_dataset = load_summary_dataset(tokenizer)

# Definir los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='results',
    num_train_epochs=10,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    save_strategy='epoch',
    logging_steps=10,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='logs'
)

# Definir el entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [6]:
trainer.train()

Step,Training Loss
10,3.983200
20,2.756700


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

KeyboardInterrupt: 

In [ ]:
# Especifica la ruta de destino donde guardar el modelo y el tokenizer
ruta_destino = 'GPT_esp_summary_v4'

# Guarda el modelo y el tokenizer en la ruta de destino
model.save_pretrained(ruta_destino)
tokenizer.save_pretrained(ruta_destino)

In [1]:
import pandas as pd
file_path = "bbc_news_es.csv"
df = pd.read_csv(file_path)[['Resumen', 'Articulos']]
df = df[list(-(df["Resumen"].duplicated()))]

df['len_articulo'] = [len(i) for i in list(df['Articulos'])]
df['len_resum'] = [len(i) for i in list(df['Resumen'])]
df

,Resumen,Articulos,len_articulo,len_resum
0,La aerolínea alemana Lufthansa puede demandar ...,Lufthansa puede demandar por la visita de Bush...,1969,1025
1,Cualquier impuesto sólo se aplicaría después d...,La UE pretende alimentar la ayuda al desarroll...,2503,1068
2,Los analistas dijeron que las cifras de Ifo y ...,La confianza de las empresas alemanas se desli...,2433,919
3,La FAO ha instado a la Organización Mundial de...,La FAO advierte sobre el impacto de las subven...,2340,879
4,"Dijo: ""Ningún país puede desarrollarse sobre l...",India busca impulsar la construcción India ha ...,2977,1141
...,...,...,...,...
1178,La Sra. Druyun mantuvo conversaciones con el S...,Ex director de Boeing recibe pena de cárcel Un...,2807,1251
1179,Verizon ha ganado una batalla de adquisición p...,Verizon'sella la adquisición de MCI' Verizon h...,2275,878
1180,El índice de gasto en consumo de la universida...,Los datos estadounidenses provocan preocupació...,3094,1302
1181,La petrolera rusa Yukos ha demandado a cuatro ...,Yukos demanda a cuatro empresas por 20 mil mil...,2906,1247


In [10]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Cargar el modelo preentrenado y el tokenizer
ruta_modelo_guardado = 'GPT_esp_summary'
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
model = GPT2LMHeadModel.from_pretrained(ruta_modelo_guardado).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(ruta_modelo_guardado)

# Función para generar resúmenes
def generate_summary(text):
    input_ids = tokenizer.encode(text, add_special_tokens=True, return_tensors='pt').to(device)
    output = model.generate(input_ids, max_length=100, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return summary

# Texto de entrada para resumir
input_text = 'La UE pretende alimentar la ayuda al desarrollo Los ministros de finanzas de la Unión Europea se reúnen el jueves para debatir propuestas, incluido un impuesto sobre el combustible para aviones, para impulsar la ayuda al desarrollo para las naciones más pobres. Los responsables políticos van a pedir un informe sobre cómo se puede recaudar más dinero para el desarrollo, dijo la UE. Los países más ricos del mundo han dicho que quieren aumentar la cantidad de ayuda que dan a 0. 7% de su renta nacional bruta anual para 2015'

# Generar el resumen
summary = generate_summary(input_text)

# Imprimir el resumen generado
print(summary)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


La UE pretende alimentar la ayuda al desarrollo Los ministros de finanzas de la Unión Europea se reúnen el jueves para debatir propuestas, incluido un impuesto sobre el combustible para aviones, para impulsar la ayuda al desarrollo para las naciones más pobres. Los responsables políticos van a pedir un informe sobre cómo se puede recaudar más dinero para el desarrollo, dijo la UE. Los países más ricos del mundo han dicho que quieren aumentar la cantidad de ayuda que dan a 0. 7% de su renta nacional bruta anual para 2015.


In [9]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Cargar el modelo preentrenado y el tokenizer
ruta_modelo_guardado = 'GPT_esp_summary_v2'
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
model = GPT2LMHeadModel.from_pretrained(ruta_modelo_guardado).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(ruta_modelo_guardado)

# Función para generar resúmenes
def generate_summary(text):
    input_ids = tokenizer.encode(text, add_special_tokens=True, return_tensors='pt').to(device)
    output = model.generate(input_ids, max_length=100, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return summary

# Texto de entrada para resumir
input_text = 'La UE pretende alimentar la ayuda al desarrollo Los ministros de finanzas de la Unión Europea se reúnen el jueves para debatir propuestas, incluido un impuesto sobre el combustible para aviones, para impulsar la ayuda al desarrollo para las naciones más pobres. Los responsables políticos van a pedir un informe sobre cómo se puede recaudar más dinero para el desarrollo, dijo la UE. Los países más ricos del mundo han dicho que quieren aumentar la cantidad de ayuda que dan a 0. 7% de su renta nacional bruta anual para 2015'

# Generar el resumen
summary = generate_summary(input_text)

# Imprimir el resumen generado
print(summary)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


La UE pretende alimentar la ayuda al desarrollo Los ministros de finanzas de la Unión Europea se reúnen el jueves para debatir propuestas, incluido un impuesto sobre el combustible para aviones, para impulsar la ayuda al desarrollo para las naciones más pobres. Los responsables políticos van a pedir un informe sobre cómo se puede recaudar más dinero para el desarrollo, dijo la UE. Los países más ricos del mundo han dicho que quieren aumentar la cantidad de ayuda que dan a 0. 7% de su renta nacional bruta anual para 2015.


In [2]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Cargar el modelo preentrenado y el tokenizer
ruta_modelo_guardado = 'GPT_esp_summary_v4'
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
model = GPT2LMHeadModel.from_pretrained(ruta_modelo_guardado).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(ruta_modelo_guardado)

# Función para generar resúmenes
def generate_summary(text):
    input_ids = tokenizer.encode(text, add_special_tokens=True, return_tensors='pt').to(device)
    output = model.generate(input_ids, max_length=100, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return summary

# Texto de entrada para resumir
input_text = 'La UE pretende alimentar la ayuda al desarrollo Los ministros de finanzas de la Unión Europea se reúnen el jueves para debatir propuestas, incluido un impuesto sobre el combustible para aviones, para impulsar la ayuda al desarrollo para las naciones más pobres. Los responsables políticos van a pedir un informe sobre cómo se puede recaudar más dinero para el desarrollo, dijo la UE. Los países más ricos del mundo han dicho que quieren aumentar la cantidad de ayuda que dan a 0. 7% de su renta nacional bruta anual para 2015'

# Generar el resumen
summary = generate_summary(input_text)

# Imprimir el resumen generado
print(summary)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


La UE pretende alimentar la ayuda al desarrollo Los ministros de finanzas de la Unión Europea se reúnen el jueves para debatir propuestas, incluido un impuesto sobre el combustible para aviones, para impulsar la ayuda al desarrollo para las naciones más pobres. Los responsables políticos van a pedir un informe sobre cómo se puede recaudar más dinero para el desarrollo, dijo la UE. Los países más ricos del mundo han dicho que quieren aumentar la cantidad de ayuda que dan a 0. 7% de su renta nacional bruta anual para 2015.


In [3]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Cargar el modelo preentrenado y el tokenizer
ruta_modelo_guardado = 'GPT_esp_summary_v4'
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
model = GPT2LMHeadModel.from_pretrained(ruta_modelo_guardado).to(device)
tokenizer = GPT2Tokenizer.from_pretrained(ruta_modelo_guardado)

# Función para generar resúmenes
def generate_summary(text):
    input_ids = tokenizer.encode(text, add_special_tokens=True, return_tensors='pt').to(device)
    output = model.generate(input_ids, max_length=100, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return summary

# Texto de entrada para resumir
input_text = 'Que día es hoy? Es lunes creo'
# 'El Ministerio de Transportes y Comunicaciones a través de la Resolución directoral 40-2021, publicada hoy en el Diario El Peruano, decidió ampliar la vigencia de los certificados de salud para el otorgamiento de las licencias. Se extiende hasta el 31 de julio, las licencias de conducir de la clase B y los certificados de salud para el otorgamiento de las licencias de conducir de la clase B, cuyo vencimiento se haya producido desde el 1 de enero de 2020 hasta el 30 de julio'

# Generar el resumen
summary = generate_summary(input_text)

# Imprimir el resumen generado
print(summary)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Que día es hoy? Es lunes creo que es el día más feliz de mi vida y no puedo pensar en otra cosa que hacer", dijo el Sr. Dowell.. El Sr. Dowell dijo que el gobierno estaba considerando la posibilidad de reducir el número de personas afectadas por el tsunami.. El Sr. Dowell dijo que el gobierno estaba considerando la posibilidad de reducir el número de personas afectadas por el tsunami.. El Sr. Dowell dijo que el gobierno estaba considerando la posibilidad de
